In [1]:
import matplotlib
from datascience import Table
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
plt.style.use('fivethirtyeight')

from datascience.predicates import are
import subprocess
import os

In [20]:
def total_cost(cost_table):
    return sum(cost_table.apply(lambda row: row[4]))

In [37]:
costs_1 = Table.read_table('costs/outputs.csv')
costs_2 = Table.read_table('costs/outputs-concorde.csv')

In [38]:
print(total_cost(costs_1))
print(total_cost(costs_2))

2397365144778.0933
1952293876353.008


In [23]:
costs_1.where('output_filename', are.above('720.out'))

output_dir,output_filename,travel_cost,conquer_cost,cost
outputs,721.out,80,61,141
outputs,722.out,80,60,140
outputs,723.out,656.773,589.814,1246.59
outputs,724.out,706.375,680.578,1386.95
outputs,725.out,459.672,492.71,952.382
outputs,726.out,4.1539e+07,1.60577e+07,5.75967e+07
outputs,727.out,9.33082e+07,3.45869e+07,1.27895e+08
outputs,728.out,1.38452e+10,2.32112e+09,1.61663e+10
outputs,729.out,6417,4018,10435
outputs,73.out,360,320,680


In [24]:
costs_2

output_dir,output_filename,travel_cost,conquer_cost,cost
outputs-concorde,0.out,28,40,68
outputs-concorde,1.out,194,52,246
outputs-concorde,10.out,175.69,281.31,457
outputs-concorde,100.out,403864,27704,431568
outputs-concorde,105.out,1.16009e+07,1.80057e+07,2.96065e+07
outputs-concorde,106.out,2.53959e+07,3.52362e+07,6.06321e+07
outputs-concorde,107.out,6.44545e+06,6.61577e+06,1.30612e+07
outputs-concorde,108.out,6936.75,2278.12,9214.87
outputs-concorde,109.out,5922.62,1758.54,7681.17
outputs-concorde,11.out,105.88,102.89,208.77


In [25]:
d = costs_1.join('output_filename', costs_2)
d.where("cost", are.above, "cost_2")
# d

output_filename,output_dir,travel_cost,conquer_cost,cost,output_dir_2,travel_cost_2,conquer_cost_2,cost_2
10.out,outputs,250.8,281.31,532.11,outputs-concorde,175.69,281.31,457
105.out,outputs,1.78909e+07,1.80057e+07,3.58966e+07,outputs-concorde,1.16009e+07,1.80057e+07,2.96065e+07
106.out,outputs,3.59132e+07,3.52362e+07,7.11494e+07,outputs-concorde,2.53959e+07,3.52362e+07,6.06321e+07
107.out,outputs,9.28911e+06,6.61577e+06,1.59049e+07,outputs-concorde,6.44545e+06,6.61577e+06,1.30612e+07
108.out,outputs,10380,2278.12,12658.1,outputs-concorde,6936.75,2278.12,9214.87
109.out,outputs,9850.71,1758.54,11609.3,outputs-concorde,5922.62,1758.54,7681.17
11.out,outputs,141.04,102.89,243.93,outputs-concorde,105.88,102.89,208.77
110.out,outputs,8089.35,1990.7,10080,outputs-concorde,4931.54,1990.7,6922.23
111.out,outputs,1372,518,1890,outputs-concorde,1046,518,1564
112.out,outputs,2764,1050,3814,outputs-concorde,2112,1050,3162


In [27]:
appended = costs_1.copy().append(costs_2).sort('output_filename')
appended

output_dir,output_filename,travel_cost,conquer_cost,cost
outputs,0.out,16,51,67
outputs-concorde,0.out,28,40,68
outputs,1.out,194,52,246
outputs-concorde,1.out,194,52,246
outputs,10.out,250.8,281.31,532.11
outputs-concorde,10.out,175.69,281.31,457
outputs,100.out,403864,27704,431568
outputs-concorde,100.out,403864,27704,431568
outputs,101.out,324,1,325
outputs,105.out,1.78909e+07,1.80057e+07,3.58966e+07


In [28]:
with_min = appended.group('output_filename', min).select(['output_filename', 'cost min'])
with_min

output_filename,cost min
0.out,67
1.out,246
10.out,457
100.out,431568
101.out,325
105.out,2.96065e+07
106.out,6.06321e+07
107.out,1.30612e+07
108.out,9214.87
109.out,7681.17


In [29]:
min_output_selected = appended.join('output_filename', with_min).where('cost', are.equal_to, 'cost min').drop('cost min')
min_output_selected.move_to_start('output_dir')
min_output_selected

output_dir,output_filename,travel_cost,conquer_cost,cost
outputs,0.out,16,51,67
outputs,1.out,194,52,246
outputs-concorde,1.out,194,52,246
outputs-concorde,10.out,175.69,281.31,457
outputs,100.out,403864,27704,431568
outputs-concorde,100.out,403864,27704,431568
outputs,101.out,324,1,325
outputs-concorde,105.out,1.16009e+07,1.80057e+07,2.96065e+07
outputs-concorde,106.out,2.53959e+07,3.52362e+07,6.06321e+07
outputs-concorde,107.out,6.44545e+06,6.61577e+06,1.30612e+07


In [30]:
# Remove duplicates
rows_to_take = []
seen = set()
for i in range(min_output_selected.num_rows):
    filename = min_output_selected.row(i)[1]
    if filename not in seen:
        seen.add(filename)
        rows_to_take.append(i)
min_output_selected = min_output_selected.take(rows_to_take)
min_output_selected

output_dir,output_filename,travel_cost,conquer_cost,cost
outputs,0.out,16,51,67
outputs,1.out,194,52,246
outputs-concorde,10.out,175.69,281.31,457
outputs,100.out,403864,27704,431568
outputs,101.out,324,1,325
outputs-concorde,105.out,1.16009e+07,1.80057e+07,2.96065e+07
outputs-concorde,106.out,2.53959e+07,3.52362e+07,6.06321e+07
outputs-concorde,107.out,6.44545e+06,6.61577e+06,1.30612e+07
outputs-concorde,108.out,6936.75,2278.12,9214.87
outputs-concorde,109.out,5922.62,1758.54,7681.17


In [31]:
# To implement: select files from min_out_selected and copy into a new folder
new_outputs_name = "outputs-concorde-2"

if not os.path.exists(new_outputs_name):
    os.makedirs(new_outputs_name)

for i in range(min_output_selected.num_rows):
    row = min_output_selected.row(i)
    dirname = row[0]
    filename = row[1]
    subprocess.run(['cp', f'{dirname}/{filename}', f'{new_outputs_name}/{filename}'])

print(total_cost(min_output_selected))
# min_output_selected.to_csv(f'costs/{new_outputs_name}.csv')

1944953182510.4285


In [32]:
2397365144778.0933 - 1944953182510.4285

452411962267.6648